In [1]:
import pandas as pd
import numpy as np
import re
import json_lines
import os
import gzip
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from scipy import spatial

In [2]:
def load_data(path):
    instance_file = open(os.path.join(path, 'instances.jsonl'),'rb')
    truth_file = open(os.path.join(path, 'truth.jsonl'),'rb')
    j_instance = []
    j_truth = []
    for item in json_lines.reader(instance_file):
        j_instance.append(item)
    
    for item in json_lines.reader(truth_file):
        j_truth.append(item)
    
    df_instance = pd.DataFrame(j_instance)
    df_truth = pd.DataFrame(j_truth)
    instance_file.close()
    truth_file.close()
    return df_instance, df_truth

In [3]:
%%time
df_instance , df_truth = load_data('clickbait17-validation-170630/')

FileNotFoundError: [Errno 2] No such file or directory: 'clickbait17-validation-170630/instances.jsonl'

In [4]:
def preprocess(text, ps):
    filtered_words = [ps.stem(word) for word in words if word not in stopwords.words('english')]

In [5]:
df_instance = df_instance.drop('postMedia',1)
ps = PorterStemmer()

NameError: name 'df_instance' is not defined

In [ ]:
df_instance.head()

In [ ]:
df_truth[df_truth['id']=='858323428260139008']
df_truth.head()

In [ ]:
def loadGloveModel(File):
    print("Loading Glove Model")
    f = open(File,'r',encoding="utf8")
    gloveModel = {}
    for line in f:
        splitLines = line.split()
        word = splitLines[0]
        wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        gloveModel[word] = wordEmbedding
    print(len(gloveModel)," words loaded!")
    return gloveModel


In [ ]:
embeddings = loadGloveModel(r'glove.twitter.27B/glove.twitter.27B.100d.txt')

In [ ]:
np.save('word_embeddings_dict.npy', embeddings) 

In [ ]:
embedding_index = np.load('word_embeddings_dict.npy',allow_pickle='TRUE').item()

In [ ]:
embedding_index['hello']

In [ ]:
df= df_instance.copy()


In [ ]:
df['postText'] = [re.sub('[^a-zA-Z0-9 ]','',str(x)) for x in df['postText']]
df['postText'] = [re.sub('xa0',' ',str(x)) for x in df['postText']]


In [ ]:
df['targetParagraphs'] = [re.sub('[^a-zA-Z0-9 ]','',str(x)) for x in df['targetParagraphs']]
df['targetParagraphs'] = [re.sub('xa0',' ',str(x)) for x in df['targetParagraphs']]

In [ ]:
df['targetTitle'] = [re.sub('[^a-zA-Z0-9 ]',' ',str(x)) for x in df['targetTitle']]
df['targetTitle'] = [re.sub('xa0',' ',str(x)) for x in df['targetTitle']]
df.iloc[0]['targetTitle']

In [ ]:
df['targetDescription'] = [re.sub('[^a-zA-Z0-9 ]',' ',str(x)) for x in df['targetDescription']]
df['targetDescription'] = [re.sub('xa0',' ',str(x)) for x in df['targetDescription']]
df.iloc[0]['targetDescription']

In [ ]:
df['Day'] = [x[:3] for x in df['postTimestamp']]

In [ ]:
df['Hour'] = [int(x[11:13]) for x in df['postTimestamp']]

In [ ]:
df.to_csv('cleaned_clickbait.csv',index=False)

In [ ]:
def doc_embedding(text):
    text=str(text)
    tokens = word_tokenize(text.lower())
    word_matrix = np.empty((1,100))
    for i in tokens:
        try:
            word_embed = np.array(embedding_index[i]).reshape((1,100))
            word_matrix = np.append(word_matrix,word_embed,axis=0)
            #print(i)
        except:
            pass
    sentence_embed = np.mean(word_matrix,axis=0).reshape(1,100)
    return sentence_embed
    

In [ ]:
#df=pd.read_csv('cleaned_clickbait.csv')ClickbaitCombined
df=pd.read_csv('ClickbaitCombined.csv')

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
%%time
df['postTextEmbed'] = [doc_embedding(text)[0] for text in df['postText']]

In [ ]:
%%time
df['targetParagraphsEmbed'] = [doc_embedding(text)[0] for text in df['targetParagraphs']]

In [ ]:
%%time
df['targetTitleEmbed'] = [doc_embedding(text)[0] for text in df['targetTitle']]

In [ ]:
%%time
df['targetDescriptionEmbed'] = [doc_embedding(text)[0] for text in df['targetDescription']]

In [ ]:
#df.to_csv('cleaned_clickbait.csv',index=False)
#df.to_csv('ClickbaitCombined.csv',index=False)

In [ ]:
def time_class(time):
    if time >= 4 and time <12:
        return 'Morning'
    elif time >= 12 and time <17:
        return 'Afternoon'
    elif time >= 17 and time <20:
        return 'Evening'
    else:
        return 'Night'

In [ ]:
%%time
df['TimeClass'] = [time_class(hour) for hour in df['Hour']]

In [ ]:
np.save('postTextEmbed.npy', df['postTextEmbed']) 

In [ ]:
np.save('targetParagraphsEmbed.npy', df['targetParagraphsEmbed']) 

In [ ]:
np.save('targetTitleEmbed.npy', df['targetTitleEmbed']) 

In [ ]:
np.save('targetDescriptionEmbed.npy', df['targetDescriptionEmbed']) 

In [ ]:
df['postTextEmbed'] = np.load('postTextEmbed.npy',allow_pickle='TRUE')

In [ ]:
df['targetParagraphsEmbed'] = np.load('targetParagraphsEmbed.npy',allow_pickle='TRUE')

In [ ]:
df['targetTitleEmbed'] = np.load('targetTitleEmbed.npy',allow_pickle='TRUE')

In [ ]:
df['targetDescriptionEmbed'] = np.load('targetDescriptionEmbed.npy',allow_pickle='TRUE')

In [ ]:
df_truth.to_csv('df_truth',index=False)

In [ ]:
df1 =df.copy()

In [ ]:
df2 = pd.merge(df1,df_true,left_on='id',right_on='id')

In [ ]:
df2.to_csv('ClickbaitCombined.csv',index=False)

In [ ]:
df.isnull().sum()

In [ ]:
nan_rows = df[df['targetDescription'].isnull()]
nan_rows.head()

In [ ]:
df_instance.iloc[23]

In [ ]:
def embed_cosine(col1,col2):
    cos_similaritycol = []
    for i in range(len(col1)):
        #print(i)
        val1 = col1.iloc[i]
        val2 = col2.iloc[i]
        similarity = 1 - spatial.distance.cosine(val1, val2)
        cos_similaritycol.append(similarity)
    return cos_similaritycol

In [ ]:
%%time
df['postText_Paragraph_Similarity'] = embed_cosine(df['postTextEmbed'],df['targetParagraphsEmbed'])
df['postText_Title_Similarity'] = embed_cosine(df['postTextEmbed'],df['targetTitleEmbed'])
df['postText_Description_Similarity'] = embed_cosine(df['postTextEmbed'],df['targetDescriptionEmbed'])
df['Paragraph_Title_Similarity'] = embed_cosine(df['targetParagraphsEmbed'],df['targetTitleEmbed'])
df['Paragraph_Description_Similarity'] = embed_cosine(df['targetParagraphsEmbed'],df['targetDescriptionEmbed'])
df['Title_Description_Similarity'] = embed_cosine(df['targetTitleEmbed'],df['targetDescriptionEmbed'])

In [ ]:
df.to_csv('ClickbaitCombined.csv',index=False)

In [ ]:
def stacked_embeds(col):
    stacked_embeds = []
    for i in col:
        stacked_embeds.append(i)
    stacked_embeds = np.asarray(stacked_embeds)
    return stacked_embeds

In [ ]:
title_stacked_embeds = stacked_embeds(df['targetTitleEmbed'])
title_stacked_embeds.shape

In [ ]:
np.save('Title Stacked Embeds.npy', title_stacked_embeds) 

In [ ]:
title_stacked_embeds = np.load('Title Stacked Embeds.npy',allow_pickle='TRUE')

In [ ]:
title_stacked_embeds.shape

In [ ]:
paragraph_stacked_embeds = stacked_embeds(df['targetParagraphsEmbed'])
paragraph_stacked_embeds.shape

In [ ]:
np.save('Paragraph Stacked Embeds.npy', paragraph_stacked_embeds) 

In [ ]:
df.columns

In [ ]:
paragraph_stacked_embeds = np.load('Paragraph Stacked Embeds.npy',allow_pickle='TRUE')

In [ ]:
#X = np.append(title_stacked_embeds,paragraph_stacked_embeds,axis=1)
X = df[['postText_Paragraph_Similarity',
'postText_Title_Similarity',
'postText_Description_Similarity',
'Paragraph_Title_Similarity',
'Paragraph_Description_Similarity',
'Title_Description_Similarity']]

In [ ]:
df=pd.read_csv('ClickbaitCombined.csv')

In [ ]:
y = df['truthClass']

In [ ]:
%%time
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=1)
sss.get_n_splits(X, y)
for train_index, test_index in sss.split(X, y):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Define the parameters to tune
parameters = { 
    'kernel' : ('linear','poly','rbf','sigmoid'),
    'C': [0.01,0.1,1,10]
}
# Tune yyperparameters  using Grid Search and a SVM model
#model = GridSearchCV(SVC(), param_grid = parameters, cv=10, n_jobs=-1,scoring='accuracy').fit(X_train, y_train)
model = SVC(kernel='poly',C=0.1).fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
print("train score:", model.score(X_train, y_train))
print("test score:", model.score(X_test, y_test))

In [ ]:
model.best_params_
# C=10 and kernel ='poly'

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
clf = RandomForestClassifier(n_estimators=400,max_depth=15)
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
print("train score:", clf.score(X_train, y_train))
print("test score:", clf.score(X_test, y_test))

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression

parameters ={'C':[0.01,0.1,1,10.0]}
#lr = GridSearchCV(LogisticRegression(max_iter=10000), param_grid = parameters, cv=10, n_jobs=-1,scoring='accuracy').fit(X_train, y_train)
lr = LogisticRegression(C=10,max_iter=20000).fit(X_train, y_train)

In [ ]:
lr.best_params_

In [ ]:
y_pred = lr.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
print("train score:", lr.score(X_train, y_train))
print("test score:", lr.score(X_test, y_test))

In [ ]:
%%time
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators=100, random_state=0)
ada.fit(X_train, y_train)

In [ ]:
y_pred = ada.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
print("train score:", ada.score(X_train, y_train))
print("test score:", ada.score(X_test, y_test))

In [ ]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=50, learning_rate=1.0,
      max_depth=5, random_state=0).fit(X_train, y_train)


In [ ]:
y_predg = gb.predict(X_test)

In [ ]:
print(classification_report(y_test,y_predg))

In [ ]:
print("train score:", gb.score(X_train, y_train))
print("test score:", gb.score(X_test, y_test))

In [ ]:
import xgboost as xgb
xgbmodel = xgb.XGBClassifier( max_depth=5,learning_rate=0.1,n_estimators=50,random_state=1,objective='binary:logistic')
xgbmodel.fit(X_train,y_train)


In [ ]:
ypred = xgbmodel.predict(X_test)

In [ ]:
print(classification_report(y_test,ypred))

In [ ]:
print("train score:", xgbmodel.score(X_train, y_train))
print("test score:", xgbmodel.score(X_test, y_test))